In [10]:
import numpy as np
import pandas as pd
import os
#from pandas.io.data import DataReader

#### 파일 불러오기

In [11]:
data_dir = 'D:/Dropbox/Projects/CRAWLER/disclosure_downloader/'

In [12]:
def openFile(market_type='KOSDAQ'):
    global krx, kind, codes
    krx = pd.read_excel(os.path.join(data_dir,market_type + '_KRX.xls'))
    kind = pd.read_csv(os.path.join(data_dir,market_type + '_KIND.csv',encoding='euc-kr'))
    krx = krx[krx.columns[1:-4]]
    
    cs_krx = set(list(krx['종목코드']))
    cs_kind = set(list(kind['종목코드']))
    
    #row 갯수(회사 수)가 같은지 확인
    if len(krx) != len(kind):
        print('# of rows : KRX data and KIND data have different number of rows!!')
        return
    else:
        print('# of rows : Confirmed')
    #code 종류(속한 회사)가 같은지 확인
    if cs_krx != cs_kind:
        print('There are some problems in corporation codes!')
        return
    else:
        print('Corporation codes : Confirmed')
    
    krx = krx.sort(columns='기업명')
    krx = krx.reset_index(drop=True)
    kind = kind.sort(columns='회사명')
    kind = kind.reset_index(drop=True)
    
    if list(kind['회사명']) != list(krx['기업명']):
        print('There are some problems in corporation list!')
        return
    else:
        print('Corporation list : Confirmed\n')
        print('=========')
        print('Completed')
        print('=========')
        codes = ['0'*(6-len(str(c))) + str(c) for c in list(krx['종목코드'])]
    return

In [13]:
openFile()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Dropbox/Projects/CRAWLER/disclosure_downloader/KOSDAQ_KRX.xls'

In [7]:
KOSPI_MarketCap = 'KOSPI_MarketCap_170102'

In [8]:
cap = pd.read_csv(KOSPI_MarketCap+'.csv')
cap = cap.loc[:,['종목코드', '종목명', '현재가','시가총액','상장주식수']]
cap['현재가'] = [int(c.replace(',','')) for c in cap['현재가']]
cap['시가총액'] = [int(c.replace(',','')) for c in cap['시가총액']]
cap['상장주식수'] = [int(c.replace(',','')) for c in cap['상장주식수']]
cap['종목코드'] = ['_' + c for c in cap['종목코드']]
cap = cap.rename(columns={'현재가':'현재가_'+KOSPI_MarketCap.split('_')[-1]})

In [9]:
cap.to_csv(r'corporation_information_2016.csv', encoding='utf-8', index=False)

In [9]:
merged = pd.merge(krx, kind, on=['종목코드'])
merged['종목코드'] = codes
merged = merged.drop(['회사명', '업종_y', '상장주식수(주)', '자본금(원)'],1)
merged = merged.rename(columns={'업종_x':'업종'})
merged['업종'] = [n.replace(',', ' ') for n in list(merged['업종'])]
merged['주요제품'] = [str(n).replace(',', ' ') for n in list(merged['주요제품'])]

In [10]:
merged = pd.merge(merged, cap, on=['종목코드'])
merged['종목코드'] = ['_'+c for c in codes]

In [11]:
merged.to_csv(r'corporation_information.csv', encoding='utf-8', index=False)

In [12]:
merged.sort_index(axis=0,by='시가총액',ascending=False)[:10]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,종목코드,기업명,업종코드,업종,주요제품,상장일,결산월,종목명,현재가_181120,시가총액,상장주식수
798,_005930,삼성전자,32604,통신 및 방송 장비 제조업,IMT2000 서비스용 동기식 기지국 교환국장비 데이터단말기 동영상휴대폰 핵심칩 반...,1975-06-11,12월,삼성전자,42750,274426130925000,6419324700
169,_000660,SK하이닉스,32601,반도체 제조업,반도체 컴퓨터 통신기기 제조 도매,1996-12-26,12월,SK하이닉스,67600,49212959874000,728002365
935,_068270,셀트리온,32101,기초 의약물질 및 생물학적 제제 제조업,램시마 트룩시마 허쥬마,2018-02-09,12월,셀트리온,229000,28729454457000,125456133
109,_051910,LG화학,32001,기초 화학물질 제조업,유화/기능/합성수지 재생섬유소 산업재 리튬이온전지 평광판 PVC 제조 도매,2001-04-25,12월,LG화학,338000,23860211934000,70592343
168,_017670,SK텔레콤,106102,전기 통신업,이동전화 무선호출서비스 부가통신,1989-11-07,12월,SK텔레콤,284000,22931781924000,80745711
1922,_005490,포스코,32401,1차 철강 제조업,열연코일 냉연강판 후판 선재 스테인리스 제조,1988-06-10,12월,POSCO,257500,22450610012500,87186835
792,_207940,삼성바이오로직스,32101,기초 의약물질 및 생물학적 제제 제조업,바이오의약품,2016-11-10,12월,삼성바이오로직스,334500,22132192500000,66165000
2155,_005380,현대자동차,33001,자동차용 엔진 및 자동차 제조업,자동차(승용차 버스 트럭 특장차) 자동차부품 자동차전착도료 제조 차량정비사업,1974-06-28,12월,현대차,97400,20811281413800,213668187
1007,_055550,신한지주,116409,기타 금융업,금융지주회사,2001-09-10,12월,신한지주,42800,20295742323600,474199587
154,_034730,SK,116409,기타 금융업,지주회사 시스템통합(SI) 정보통신사업 관련 컨설팅 조사용역 정보통신기술 연구개...,2009-11-11,12월,SK,280000,19700883160000,70360297
